# 1. process

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras_self_attention import SeqSelfAttention
import keras
import numpy as np

num_classes = 6 # classification number
x_train = np.random.randn(100, 15, 20, 3) # x_train.shape = (100, 15, 20, 3)
y_train = np.random.randint(1, size=(100,num_classes)) # y_train.shape = (100, 6)
input_shape = x_train.shape[-3:]# (15, 20, 3)


model = Sequential()
model.add(Conv2D(32, (2,11), activation='relu', padding='same', input_shape=input_shape, name='layer_conv_1'))
model.add(Conv2D(32, (2,11), activation='relu', padding='same', name='layer_conv_2'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 1), name='layer_mp_1'))
model.add(Conv2D(128, (2,7), activation='relu', padding='same', name='layer_conv_3'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 1), name='layer_mp_2'))
model.add( keras.layers.Reshape((48,128), name='layer_rsp_1') )
model.add( SeqSelfAttention( attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, name='layer_attention_1') )
model.add(Flatten(name='layer_flatten_1'))
model.add(Dense(440, activation='relu', name='layer_dense_1'))
model.add(Dropout(0.5, name='layer_dropout_1'))
model.add(Dense(num_classes, activation='softmax', name='layer_dense_2'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()                            

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_conv_1 (Conv2D)        (None, 15, 20, 32)        2144      
_________________________________________________________________
layer_conv_2 (Conv2D)        (None, 15, 20, 32)        22560     
_________________________________________________________________
layer_mp_1 (MaxPooling2D)    (None, 7, 18, 32)         0         
_________________________________________________________________
layer_conv_3 (Conv2D)        (None, 7, 18, 128)        57472     
_________________________________________________________________
layer_mp_2 (MaxPooling2D)    (None, 3, 16, 128)        0         
_________________________________________________________________
layer_rsp_1 (Reshape)        (None, 48, 128)           0         
_________________________________________________________________
layer_attention_1 (SeqSelfAt (None, 48, 128)           1

In [2]:
model.fit(x_train, y_train,batch_size=64,epochs=2,verbose=1)

Epoch 1/2
2/2 [==============================] - 1s 84ms/step - loss: 0.0000e+00
Epoch 2/2
2/2 [==============================] - 0s 92ms/step - loss: 0.0000e+00


In [3]:
import keras.backend as k

layer_name = 'layer_conv_3'
layer_output = model.get_layer(layer_name).output # get output by layer name
layer_input = model.input
output_func = k.function([layer_input], [layer_output]) # construct function

In [4]:
print( x_train.shape )
print( k.int_shape(model.input)[1:] )

(100, 15, 20, 3)
(15, 20, 3)


In [5]:
from numpy import float32

x_preproc = x_train

output_shape = output_func([x_preproc[0][None, ...]])[0].shape
activations = np.zeros((x_preproc.shape[0],) + output_shape[1:], dtype=float32)
print(output_shape)
print(activations.shape)


(1, 7, 18, 128)
(100, 7, 18, 128)


In [6]:
batch_size = 8
for batch_index in range(int(np.ceil(x_preproc.shape[0] / float(batch_size)))):
    begin, end = batch_index * batch_size, min((batch_index + 1) * batch_size, x_preproc.shape[0])
    activations[begin:end] = output_func([x_preproc[begin:end]])[0]


In [7]:
activations.shape

(100, 7, 18, 128)

In [8]:
activations

array([[[[4.14937377e-01, 6.96956992e-01, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 8.05832326e-01],
         [5.30048251e-01, 6.84755921e-01, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 8.65303457e-01],
         [4.67230588e-01, 9.54524159e-01, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 8.30184996e-01],
         ...,
         [3.08544934e-01, 6.80048048e-01, 3.17468308e-02, ...,
          0.00000000e+00, 0.00000000e+00, 9.50086534e-01],
         [2.23282963e-01, 3.65708679e-01, 1.09226309e-01, ...,
          0.00000000e+00, 0.00000000e+00, 8.28148842e-01],
         [1.06695592e-02, 2.59902775e-01, 2.36983433e-01, ...,
          0.00000000e+00, 0.00000000e+00, 4.82715487e-01]],

        [[3.62680733e-01, 6.45735860e-01, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 8.42427075e-01],
         [5.12251973e-01, 7.31132209e-01, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 1.03074324e+00],
        

# 2. the function

In [9]:
import six
import numpy as np
import keras.backend as k
from numpy import float32

def get_activations(x, model, layer, batch_size=128):
    layer_names = [layer.name for layer in model.layers]
    if isinstance(layer, six.string_types):
        if layer not in layer_names:
            raise ValueError('Layer name %s is not part of the graph.' % layer)
        layer_name = layer
    elif isinstance(layer, int):
        if layer < 0 or layer >= len(layer_names):
            raise ValueError('Layer index %d is outside of range (0 to %d included).'
                             % (layer, len(layer_names) - 1))
        layer_name = layer_names[layer]
    else:
        raise TypeError('Layer must be of type `str` or `int`.')

    layer_output = model.get_layer(layer_name).output
    layer_input = model.input
    output_func = k.function([layer_input], [layer_output])

    # Apply preprocessing
    if x.shape == k.int_shape(model.input)[1:]:
        x_preproc = np.expand_dims(x, 0)
    else:
        x_preproc = x
    assert len(x_preproc.shape) == 4

    # Determine shape of expected output and prepare array
    output_shape = output_func([x_preproc[0][None, ...]])[0].shape
    activations = np.zeros((x_preproc.shape[0],) + output_shape[1:], dtype=float32)

    # Get activations with batching
    for batch_index in range(int(np.ceil(x_preproc.shape[0] / float(batch_size)))):
        begin, end = batch_index * batch_size, min((batch_index + 1) * batch_size, x_preproc.shape[0])
        activations[begin:end] = output_func([x_preproc[begin:end]])[0]

    return activations


In [10]:
y = get_activations(x_train, model, layer='layer_dense_1', batch_size=128)
y.shape

(100, 440)